In [ ]:
import os
import sys
import subprocess

# Library installation (force Numpy < 2.0)
# Use subprocesses to ensure that it finishes before continuing
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/THU-MIG/yolov10.git"])
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "ray"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2.0", "pandas", "--force-reinstall"])

# Download the weights
if not os.path.exists('yolov10s.pt'):
    subprocess.check_call(["wget", "-q", "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt"])

In [ ]:
%%writefile train.py
import os
import glob
import random
import yaml
import torch
import numpy
from ultralytics import YOLOv10
from ultralytics import settings

# Configuration
EXPERIMENT = '100pct' #50pct, 25pct
EPOCHS = 100
BATCH_SIZE = 16

if numpy.__version__ >= "2.0.0":
    raise ImportError(" ERRORE: Numpy è ancora > 2.0!")

# WANDB setup (Disabled to prevent background crashes)
os.environ['WANDB_MODE'] = 'disabled'
settings.update({"raytune": False})

# Patch for PyTorch (weight security)
_original_load = torch.load
def safe_load(*args, **kwargs):
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _original_load(*args, **kwargs)
torch.load = safe_load

# Dataset management

# Check if it already exists
dataset_path = ''
potential_paths = [
    './datasets/VisDrone',
    '/kaggle/input/visdrone/VisDrone',  # Path if I add the dataset from Kaggle input
    '/kaggle/working/datasets/VisDrone'
]

# If there's no dataset, download YOLO dummy
if not any(os.path.exists(p) for p in potential_paths):
    try:
        model_dummy = YOLOv10('yolov10s.pt')
        model_dummy.train(data='VisDrone.yaml', epochs=1, imgsz=32, batch=1)
    except:
        print("   (Download trigger completato o interrotto)")

for path in potential_paths:
    if os.path.exists(path):
        dataset_path = path
        break

if not dataset_path:
    # Fallback: YOLO usually downloads here in /kaggle/working
    dataset_path = '/kaggle/working/datasets/VisDrone'

# Data preparation
train_images = glob.glob(os.path.join(dataset_path, 'VisDrone2019-DET-train/images/*.jpg'))
val_images = glob.glob(os.path.join(dataset_path, 'VisDrone2019-DET-val/images/*.jpg'))

if not train_images:
    raise FileNotFoundError(" ERRORE: Nessuna immagine trovata!")

random.seed(42)
random.shuffle(train_images)

n_total = len(train_images)
splits = {
    '25pct': train_images[:int(n_total * 0.25)],
    '50pct': train_images[:int(n_total * 0.5)],
    '100pct': train_images
}

selected_images = splits[EXPERIMENT]
print(f" Training su {EXPERIMENT}: {len(selected_images)} immagini.")

# Writing .txt files
cwd = os.getcwd()
visdrone_classes = {
    0: 'pedestrian', 1: 'people', 2: 'bicycle', 3: 'car', 4: 'van',
    5: 'truck', 6: 'tricycle', 7: 'awning-tricycle', 8: 'bus', 9: 'motor'
}

txt_filename = os.path.join(cwd, f'train_{EXPERIMENT}.txt')
with open(txt_filename, 'w') as f:
    f.write('\n'.join(selected_images))

val_txt = os.path.join(cwd, 'val_official.txt')
with open(val_txt, 'w') as f:
    f.write('\n'.join(val_images))

# YAML writing
yaml_content = {
    'path': cwd,
    'train': txt_filename,
    'val': val_txt,
    'names': visdrone_classes
}
yaml_filename = os.path.join(cwd, f'VisDrone_{EXPERIMENT}.yaml')
with open(yaml_filename, 'w') as f:
    yaml.dump(yaml_content, f)

# Training
model = YOLOv10('yolov10s.pt')

model.train(
    data=yaml_filename,
    epochs=EPOCHS,
    imgsz=640,
    batch=BATCH_SIZE,
    device=0,
    optimizer='AdamW',
    lr0=0.001,
    freeze=10,
    patience=10,
    project='YOLOv10_VisDrone',
    name=f'run_{EXPERIMENT}',
    exist_ok=True,
    verbose=True
)

In [ ]:
# Run the script
!python train.py

import os
import shutil

if os.path.exists("./datasets"):
    shutil.rmtree("./datasets")

# Zip creation with output
exp_name = "100pct" #50, 25
output_zip = f"risultati_yolo_VisDrone_{exp_name}.zip"

# Only zip the results folders
# (The “try-except” is needed in case one of the two folders does not exist)
folders_to_zip = f"YOLOv10_VisDrone/run_{exp_name} runs/detect/train"
os.system(f"zip -r -q {output_zip} {folders_to_zip}")

if os.path.exists(output_zip):
    print(f" Archivio pronto: {output_zip}")
else:
    print(" Attenzione: Archivio non creato.")